<a href="https://colab.research.google.com/github/auzaluis/upsa_mod_202501/blob/main/whisky/02_script_EDA_whisky.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import plotly.express as px
import re

In [2]:
path = '/content/drive/MyDrive/Teaching/UPSA/2025.01/MOD/whisky/data-whisky.parquet'

In [3]:
df = pd.read_parquet(path)

# Situación competitiva

## Lealtad

In [4]:
df_lealtad = df['¿Cuál es la marca que más compra?'] \
.value_counts() \
.reset_index() \
.rename(columns = {
    '¿Cuál es la marca que más compra?': 'Marca',
    'count': 'n'
})

In [5]:
df_lealtad['Porcentaje'] = df_lealtad['n'] / df_lealtad['n'].sum()
df_lealtad['Porcentaje_txt'] = df_lealtad['Porcentaje'].apply(lambda x: str(round(x*100, 0)) + '%')

In [6]:
df_lealtad

,Marca,n,Porcentaje,Porcentaje_txt
0,Johnny Walker,43,0.524390,52.0%
1,Jack Daniel's,18,0.219512,22.0%
2,Chivas Regal,14,0.170732,17.0%
3,Old Parr,6,0.073171,7.0%
4,Ninguno,1,0.012195,1.0%


In [ ]:
px.bar(
    df_lealtad,
    x = 'Marca',
    y = 'Porcentaje',
    text = 'Porcentaje_txt',
    title = '¿Cuál es la marca que más compra?',
    # color = 'Marca'
).update_layout(
    template = 'plotly_white',
    yaxis_showticklabels = False,
    yaxis_showgrid = False,
    xaxis_title = ''
)

## Conocimiento

In [7]:
[col for col in df.columns if col.startswith('Conocimiento')]

['Conocimiento1', 'Conocimiento2', 'Conocimiento3', 'Conocimiento4']

In [8]:
df_conocimiento_long = df.melt(
    id_vars = [],
    value_vars = [col for col in df.columns if col.startswith('Conocimiento')],
    var_name = 'KPI',
    value_name= 'Marca'
).dropna()

In [9]:
df_conocimiento_long.head()

,KPI,Marca
0,Conocimiento1,Johnny Walker
1,Conocimiento1,Johnny Walker
2,Conocimiento1,Chivas Regal
3,Conocimiento1,Chivas Regal
4,Conocimiento1,Chivas Regal


In [10]:
df_conocimiento = df_conocimiento_long['Marca'] \
.value_counts() \
.reset_index() \
.rename(columns = {
    'count': 'n'
})

In [11]:
df_conocimiento

,Marca,n
0,Johnny Walker,71
1,Chivas Regal,61
2,Jack Daniel's,56
3,Old Parr,44


In [12]:
df_conocimiento['Porcentaje'] = df_conocimiento['n'] / len(df)
df_conocimiento['Porcentaje_txt'] = df_conocimiento['Porcentaje'].apply(lambda x: str(round(x*100, 0)) + '%')

In [13]:
df_conocimiento

,Marca,n,Porcentaje,Porcentaje_txt
0,Johnny Walker,71,0.865854,87.0%
1,Chivas Regal,61,0.743902,74.0%
2,Jack Daniel's,56,0.682927,68.0%
3,Old Parr,44,0.536585,54.0%


In [ ]:
px.bar(
    df_conocimiento,
    x = 'Marca',
    y = 'Porcentaje',
    text = 'Porcentaje_txt',
    title = '¿Cuál de estas marcas conoce?',
    # color = 'Marca'
).update_layout(
    template = 'plotly_white',
    yaxis_showticklabels = False,
    yaxis_showgrid = False,
    xaxis_title = ''
)

## Prueba

In [14]:
[col for col in df.columns if col.startswith('Prueba')]

['Prueba1', 'Prueba2', 'Prueba3', 'Prueba4']

In [15]:
df_prueba_long = df.melt(
    id_vars = [],
    value_vars = [col for col in df.columns if col.startswith('Prueba')],
    var_name = 'KPI',
    value_name= 'Marca'
).dropna()

In [16]:
df_prueba = df_prueba_long['Marca'] \
.value_counts() \
.reset_index() \
.rename(columns = {
    'count': 'n'
})

In [17]:
df_prueba

,Marca,n
0,Johnny Walker,61
1,Chivas Regal,28
2,Jack Daniel's,26
3,Old Parr,24


In [83]:
df_prueba['Porcentaje'] = df_prueba['n'] / len(df)
df_prueba['Porcentaje_txt'] = df_prueba['Porcentaje'].apply(lambda x: str(round(x*100, 0)) + '%')

In [84]:
df_prueba

,Marca,n,KPI,Porcentaje,Porcentaje_txt
0,Johnny Walker,61,Prueba,0.743902,74.0%
1,Chivas Regal,28,Prueba,0.341463,34.0%
2,Jack Daniel's,26,Prueba,0.317073,32.0%
3,Old Parr,24,Prueba,0.292683,29.0%


## Consideración

In [18]:
df['Consideración Chivas'].unique()

array(['NUNCA la consideraría', 'Creo que NO la consideraría',
       'Creo que SÍ la consideraría', 'Sería mi 1ra opción'], dtype=object)

In [27]:
consideracion_t2b = ['Creo que SÍ la consideraría', 'Sería mi 1ra opción']
consideracion_t2b

['Creo que SÍ la consideraría', 'Sería mi 1ra opción']

In [20]:
consideracion_cols = [col for col in df.columns if col.startswith('Consideración')]
consideracion_cols

['Consideración Chivas',
 'Consideración Johnny Walker',
 'Consideración Old Parr',
 "Consideración Jack Daniel's"]

In [21]:
# Demo individual
df['Consideración Johnny Walker'].apply(lambda x: x in consideracion_t2b).sum()

np.int64(57)

In [22]:
# Demo individual
re.search(r'(Consideración\s)(.+)', 'Consideración Johnny Walker').group(2)

'Johnny Walker'

In [23]:
consideracion_dict = []

for col in consideracion_cols:
  Total = len(df)
  Considera = df[col].apply(lambda x: x in consideracion_t2b).sum()
  Porcentaje = Considera / Total
  Porcentaje_txt = str(round(Porcentaje*100, 0)) + '%'
  Marca = re.search(r'(Consideración\s)(.+)', col).group(2)
  consideracion_dict.append({
      'Marca': Marca,
      'n': Considera,
      'Porcentaje': Porcentaje,
      'Porcentaje_txt': Porcentaje_txt
  })

In [24]:
df_consideracion = pd.DataFrame(consideracion_dict)
df_consideracion

,Marca,n,Porcentaje,Porcentaje_txt
0,Chivas,30,0.365854,37.0%
1,Johnny Walker,57,0.695122,70.0%
2,Old Parr,34,0.414634,41.0%
3,Jack Daniel's,58,0.707317,71.0%


## Recomendación

In [30]:
recomendacion_t2b = [9,10]
recomendacion_t2b

[9, 10]

In [31]:
recomendacion_cols = [col for col in df.columns if col.startswith('Recomendación')]
recomendacion_cols

['Recomendación Chivas',
 'Recomendación Johnny Walker',
 'Recomendación Old Parr',
 "Recomendación Jack Daniel's"]

In [32]:
recomendacion_dict = []

for col in recomendacion_cols:
  Total = len(df)
  Recomienda = df[col].apply(lambda x: x in recomendacion_t2b).sum()
  Porcentaje = Recomienda / Total
  Porcentaje_txt = str(round(Porcentaje*100, 0)) + '%'
  Marca = re.search(r'(Recomendación\s)(.+)', col).group(2)
  recomendacion_dict.append({
      'Marca': Marca,
      'n': Recomienda,
      'Porcentaje': Porcentaje,
      'Porcentaje_txt': Porcentaje_txt
  })

In [33]:
df_recomendacion = pd.DataFrame(recomendacion_dict)
df_recomendacion

,Marca,n,Porcentaje,Porcentaje_txt
0,Chivas,7,0.085366,9.0%
1,Johnny Walker,14,0.170732,17.0%
2,Old Parr,4,0.048780,5.0%
3,Jack Daniel's,11,0.134146,13.0%


In [39]:
px.bar(
    df_recomendacion.sort_values('Porcentaje', ascending = False),
    x = 'Marca',
    y = 'Porcentaje',
    text = 'Porcentaje_txt',
    title = 'Promotores de marca (t2b)'
).update_layout(
    template = 'simple_white',
    showlegend = False,
    yaxis_showticklabels = False,
    yaxis_ticks = '',
    xaxis_ticks = '',
    xaxis_title = None
)

### Stacked Bar Chart

In [40]:
nps_cols = [col for col in df.columns if col.startswith('Recomendación')]
nps_cols

['Recomendación Chivas',
 'Recomendación Johnny Walker',
 'Recomendación Old Parr',
 "Recomendación Jack Daniel's"]

In [41]:
df['Recomendación Chivas'].unique()

array([ 0,  2,  9,  4,  5, 10,  3,  6,  8,  7,  1], dtype=int32)

In [54]:
df['Recomendación Chivas'].apply(lambda x: 'Promotor' if x >= 9 else ('Neutro' if x >= 7 else 'Detractor')).value_counts().get('Detractor', 0)

np.int64(66)

In [52]:
nps_cols

['Recomendación Chivas',
 'Recomendación Johnny Walker',
 'Recomendación Old Parr',
 "Recomendación Jack Daniel's"]

In [57]:
nps_dict = []

for col in nps_cols:
  Marca = re.search(r'(Recomendación\s)(.+)', col).group(2)
  clasificacion = df[col].apply(lambda x: 'Promotor' if x >= 9 else ('Neutro' if x >= 7 else 'Detractor'))
  conteo = clasificacion.value_counts()
  proporcion = clasificacion.value_counts(normalize=True)

  for tipo in ['Promotor', 'Neutro', 'Detractor']:
    n = conteo.get(tipo, 0)
    Porcentaje = proporcion.get(tipo, 0)
    Porcentaje_txt = str(round(Porcentaje*100, 0)) + '%'
    nps_dict.append({
        'Marca': Marca,
        'Tipo': tipo,
        'n': n,
        'Porcentaje': Porcentaje,
        'Porcentaje_txt': Porcentaje_txt
    })

In [60]:
df_nps = pd.DataFrame(nps_dict)
df_nps

,Marca,Tipo,n,Porcentaje,Porcentaje_txt
0,Chivas,Promotor,7,0.085366,9.0%
1,Chivas,Neutro,9,0.109756,11.0%
2,Chivas,Detractor,66,0.804878,80.0%
3,Johnny Walker,Promotor,14,0.170732,17.0%
4,Johnny Walker,Neutro,14,0.170732,17.0%
5,Johnny Walker,Detractor,54,0.658537,66.0%
6,Old Parr,Promotor,4,0.048780,5.0%
7,Old Parr,Neutro,13,0.158537,16.0%
8,Old Parr,Detractor,65,0.792683,79.0%
9,Jack Daniel's,Promotor,11,0.134146,13.0%


In [78]:
px.bar(
    df_nps,
    x = 'Marca',
    y = 'Porcentaje',
    text = 'Porcentaje_txt',
    color = 'Tipo',
    title = 'Net Promoter Score',
    category_orders = dict(Tipo = ['Detractor', 'Neutro', 'Promotor']),
    color_discrete_map = dict(
        Detractor = '#f7c59f',
        Neutro = '#efefd0',
        Promotor = '#1a659e'
    )
).update_layout(
    template = 'simple_white',
    yaxis_showticklabels = False,
    yaxis_ticks = '',
    xaxis_ticks = '',
    xaxis_title = None,
    legend = dict(orientation = 'h', y = 1.1, traceorder = 'reversed')
)

# Brand Funnel

In [86]:
df_conocimiento['KPI'] = 'Conocimiento'
df_consideracion['KPI'] = 'Consideración'
df_prueba['KPI'] = 'Prueba'
df_lealtad['KPI'] = 'Lealtad'
df_recomendacion['KPI'] = 'Recomendación'

In [87]:
df_funnel = pd.concat([
    df_conocimiento,
    df_consideracion,
    df_prueba,
    df_lealtad,
    df_recomendacion
])

In [88]:
df_funnel

,Marca,n,Porcentaje,Porcentaje_txt,KPI
0,Johnny Walker,71,0.865854,87.0%,Conocimiento
1,Chivas Regal,61,0.743902,74.0%,Conocimiento
2,Jack Daniel's,56,0.682927,68.0%,Conocimiento
3,Old Parr,44,0.536585,54.0%,Conocimiento
0,Chivas,30,0.365854,37.0%,Consideración
1,Johnny Walker,57,0.695122,70.0%,Consideración
2,Old Parr,34,0.414634,41.0%,Consideración
3,Jack Daniel's,58,0.707317,71.0%,Consideración
0,Johnny Walker,61,0.743902,74.0%,Prueba
1,Chivas Regal,28,0.341463,34.0%,Prueba
